# Human in the loop

Human-in-the-loop (HITL) patterns for AI agents fall into a few broad categories:

### 1. Approval Gates
The agent pauses before executing high-stakes or irreversible actions, waiting for human confirmation:

- Financial transactions or purchases
- Contractual agreement execution
- Sending emails/messages on behalf of the user
- Modifying production systems or databases
- Deleting or overwriting data

###  2. Ambiguity Resolution
The agent encounters uncertainty and escalates to a human for clarification rather than guessing:

- Underspecified user intent ("schedule a meeting" — with whom? when?)
- Multiple valid interpretations of a request
- Missing context the agent can't infer
- Edge cases not covered by the agent's instructions

### 3. Quality Assurance
Human reviews agent outputs before they're finalized or delivered:

- Reviewing generated content before publishing
- Validating research findings or summaries
- Checking code changes before committing
- Approving customer-facing responses

### 4. Error Recovery / Correction
Human intervention when the agent gets stuck or makes mistakes:

- Agent hits a dead end and needs guidance
- Correcting factual errors or hallucinations
- Recovering from tool failures
- Redirecting when the agent misunderstands the goal

### 5. Policy/Judgment Calls
Decisions that require human values, ethics, or domain expertise the agent shouldn't make autonomously:

- Prioritization decisions with competing stakeholders
- Situations with legal or compliance implications
- Novel situations outside the agent's training distribution
- Anything requiring organizational authority


## Costs of HITL interrupts:
Human in the loop interrupts must be used thoughtful to not destroy productivity gains:

- **Latency** — Human response time becomes the bottleneck; defeats the point if you're waiting hours for approval on routine tasks
- **Scalability**  — Can't run N agents in parallel if each needs human attention; you become the limiting factor
- **Context switching** — Humans get interrupt fatigue, make worse decisions when constantly pulled in
- **Defeats autonomy** — Too many checkpoints and you've just built a fancy todo list, not an agent

# Mock e-mail agent

In [ ]:
from datetime import datetime
from pprint import pprint
from typing import Any, Literal

from diskcache import Cache
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage
from langchain.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command
from loguru import logger
from pydantic import BaseModel, EmailStr, Field

from chain_reaction.caching import cache_calls
from chain_reaction.config import APIKeys, ModelBehavior, ModelName
from chain_reaction.utils import get_last_message, get_structured_response

api_keys = APIKeys()

# Initialize temporary cache
cache = Cache()

# Initialize a chat model
chat_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
    **ModelBehavior.deterministic().model_dump(),
)

In [ ]:
# Email data model
class Email(BaseModel):
    """Email representation."""

    sender: EmailStr = Field(description="The email address of the sender.")
    recipient: EmailStr = Field(description="The email address of the recipient.")
    timestamp: datetime | None = Field(description="The timestamp of the email. If None, email has not been sent yet.")
    subject: str = Field(description="The subject of the email.")
    body: str = Field(description="The body content of the email.")

    @classmethod
    def get_result(cls, response: dict[str, Any]) -> Email:
        """Parse Email from response dictionary."""
        result = get_structured_response(response, cls)
        if result is None:
            raise ValueError("Failed to parse Email from response.")
        return result


# Subagent for composing emails
email_composition_agent = create_agent(
    model=chat_model,
    system_prompt="""
    You're an email composition assistant.
    Help the user draft professional and clear emails based on their instructions.
    """,
    response_format=Email,
)

In [ ]:
# Define mock email tools
@tool
def get_emails() -> list[Email]:
    """Retrieve emails from inbox.

    Returns:
        list[Email]: A list of emails in the inbox.
    """
    # Mock inbox
    # Replace this with actual email retrieval logic
    inbox = [
        Email(
            sender="test@sender1.com",
            recipient="nick@recipient.com",
            timestamp=datetime(2025, 12, 30, 14, 30),
            subject="Need to reschedule our meeting",
            body="""
            Hi Nick,
            I need to reschedule our meeting. Can we find another time that works for both of us?
            Best, Julia
            """,
        ),
        Email(
            sender="test@sender2.com",
            recipient="nick@recipient.com",
            timestamp=datetime(2025, 12, 30, 7, 21),
            subject="Order Shipment Confirmation",
            body="Hello Nick, Your order of '50 gallon goldfish snacks' has been shipped. Thanks, Alex",
        ),
        Email(
            sender="test@sender1.com",
            recipient="nick@recipient.com",
            timestamp=datetime(2025, 12, 30, 6, 58),
            subject="🗓️ Reschedule your haircut! ✂️",
            body="Hi Nick, Please reschedule your haircut appointment to next week. Thanks, AI Cuts ✂️",
        ),
    ]

    logger.debug(f"Retrieved {len(inbox)} emails from inbox.")

    return inbox


@tool
@cache_calls(cache)
def decide_if_email_needs_response(email: Email) -> bool:
    """Decide if an email needs a response.

    Args:
        email (Email): The email to evaluate.

    Returns:
        bool: True if the email needs a response, False otherwise.
    """
    # Mock logic to decide if an email needs a response
    # Replace this with actual decision-making logic
    needs_response = "reschedule" in email.subject.lower()

    logger.info(f"Email from {email.sender} with subject '{email.subject}' needs response: {needs_response}")

    return needs_response


@tool
@cache_calls(cache)
def compose_response_email(original_email: Email) -> Email:
    """Invoke email composition agent to draft a response email.

    Args:
        original_email (Email): The original email to respond to.

    Returns:
        Email: The composed response email.
    """
    # Invoke the email composition agent to draft a response
    response = email_composition_agent.invoke(
        {
            "messages": [
                HumanMessage(
                    content=f"Please draft a response to the following email:\n\n{original_email.model_dump_json()}"
                )
            ]
        },
    )

    # Parse the response into an Email object
    response_email = Email.get_result(response)

    logger.info(f"Composed response email to {response_email.recipient} with subject: {response_email.subject}")

    return response_email


@tool
def send_email(email: Email) -> None:
    """Send an email.

    Args:
        email (Email): The email to send.
    """
    # Logic to send email defined here
    # This is a mock implementation
    logger.info(f"Sending email to {email.recipient} with subject: {email.subject}")

In [ ]:
# Define function for human-in-the-loop interrupt handling
type Decision = Literal["approve", "reject"]


def approve_or_reject(decisions: list[tuple[Decision, str]]) -> Command:
    """Generate a command based on human approval or rejection.

    Args:
        decisions (list[tuple[Decision, str]]): A list of decisions made by the human.
            Each decision is a tuple containing the decision ("approve" or "reject") and an explanation.

    Returns:
        Command: A command object representing the human's decisions.
    """
    return Command(
        resume={"decisions": [{"type": decision, "message": explanation} for decision, explanation in decisions]}
    )

In [ ]:
# Define email agent with human-in-the-loop middleware
email_agent = create_agent(
    model=chat_model,
    system_prompt="""
    You're an email management assistant.

    Help the user manage their emails by:
    1. retrieving emails from their inbox,
    2. For each email, deciding if it needs a response: (True/False),
    3. For each email that needs a response, composing a response email,
    4. Send each composed email.

    Use the provided tools to perform actions.
    """,
    tools=[get_emails, decide_if_email_needs_response, compose_response_email, send_email],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "get_emails": False,
                "decide_if_email_needs_response": False,
                "compose_response_email": False,
                "send_email": True,
            },
        )
    ],
)

In [ ]:
# Invoke the email agent
config = {"configurable": {"thread_id": "1"}}
response = email_agent.invoke({"messages": HumanMessage("Can you help me with my emails?")}, config=config)

In [ ]:
# Check for interrupt in response and print action request description
if "__interrupt__" in response:
    action_requests = response["__interrupt__"][0].value["action_requests"]
    print(f"{len(action_requests)} Action Requests:")
    for action_request in action_requests:
        pprint(action_request["description"])
else:
    pprint(get_last_message(response))

In [ ]:
# Respond to the first interrupt by approving the email
decision_1 = ("approve", "The drafted email looks good.")

# Respond to the second interrupt by rejecting the email and providing feedback
decision_2 = ("reject", "Propose 1/3/2026 at 3pm.")

# Continue the interaction with both decisions
response = email_agent.invoke(
    approve_or_reject([decision_1, decision_2]),
    config=config,  # Pass the same config to maintain thread ID
)

In [ ]:
# Check for interrupt in response and print action request description
if "__interrupt__" in response:
    action_requests = response["__interrupt__"][0].value["action_requests"]
    print(f"{len(action_requests)} Action Requests:")
    for action_request in action_requests:
        pprint(action_request["description"])
else:
    pprint(get_last_message(response).content)

In [ ]:
# If the agent didn't loop back around with an edit to the rejected email, prompt it again
response = email_agent.invoke(
    {
        "messages": HumanMessage(
            "I don't think you sent the email about rescheduling my haircut. Please propose 1/3/2026 at 3pm."
        )
    },
    config=config,
)

# Check for interrupt in response and print action request description
if "__interrupt__" in response:
    action_requests = response["__interrupt__"][0].value["action_requests"]
    print(f"{len(action_requests)} Action Requests:")
    for action_request in action_requests:
        pprint(action_request["description"])
else:
    pprint(get_last_message(response).content)

In [ ]:
# Approve the edited email
response = email_agent.invoke(
    approve_or_reject([("approve", "The revised email looks good.")]),
    config=config,  # Pass the same config to maintain thread ID
)

# Now, there should be no more interrupts; print final message
assert "__interrupt__" not in response  # noqa: S101
pprint(get_last_message(response).content)